In [36]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

In [37]:
# Supress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [38]:
# Fungsi untuk plot kurva akurasi dan loss
def plot_model_history(model_history):
    """
    Plot kurva Akurasi dan Loss dari model_history.
    """
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    # Plot Akurasi
    axs[0].plot(range(1, len(model_history.history['accuracy']) + 1), model_history.history['accuracy'])
    axs[0].plot(range(1, len(model_history.history['val_accuracy']) + 1), model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['Train', 'Validation'], loc='best')

    # Plot Loss
    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['Train', 'Validation'], loc='best')

    plt.tight_layout()
    plt.show()

In [39]:
# Path direktori dataset
train_dir = 'data/train'
val_dir = 'data/test'

In [40]:
# Augmentasi dataset menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [41]:
# Membuat model
model = Sequential([
    Input(shape=(48, 48, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 kelas untuk pengenalan emosi
])

In [42]:
# Callback untuk optimisasi pelatihan
callbacks = [
    ModelCheckpoint('best_emotion_recognition_model.keras', save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
]

In [43]:
# Kompilasi model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [44]:
# Hitung steps_per_epoch secara dinamis
steps_per_epoch = (train_generator.samples + batch_size - 1) // batch_size
validation_steps = (validation_generator.samples + batch_size - 1) // batch_size

# Pelatihan model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=num_epoch,
    callbacks=callbacks
)

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 193s 427ms/step - accuracy: 0.2423 - loss: 1.8338 - val_accuracy: 0.2483 - val_loss: 1.8122 - learning_rate: 1.0000e-04
Epoch 2/50


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# Plot kurva hasil pelatihan
plot_model_history(model_info)

In [ ]:
# Menyimpan model lengkap (arsitektur + bobot + konfigurasi)
model.save('final_emotion_recognition_model.h5')